In [1]:
import numpy as np
import pandas as pd
import math



In [2]:
def gaze_angle(x, y):
        angle = (math.atan(x/y)/math.pi)*180
        return angle

def data_org(df):
    df = df.drop(df[df.BPOGV < 1].index) #remove invalid eyetracking data
    #outlier filtering 
    df = df.loc[(df['BPOGX'] >= 0) & (df['BPOGX'] <= 1)]
    df = df.loc[(df['BPOGY'] >= 0) & (df['BPOGY'] <= 1)]
    df = df.reset_index(drop = True)

    df['gaze_angle'] = list(map(gaze_angle, df['BPOGX'], df['BPOGY']))

    df['dif'] = abs(df['gaze_angle'].diff(periods=-1))
    return df
    
def fixation_detection(df, missing = 0.0, maxdist =1, mindur = 0.1):
    '''
    Sfix-list of lists, each containing [starttime]
    Efix-list of lists, each containing [starttime, endtime, duration, endx, endy]
    '''
    Sfix = []
    Efix = []
    x = df['gaze_angle']
    time = df['timestamp']
    block = df['block']
    nback = df['nback']
    target = df['target']
    zone = df['zone']
    stimuli = df['stimuli']

    bpx = df['BPOGX']
    bpy = df['BPOGY']
    #info = df[['block', 'nback', 'target', 'zone' , 'stimuli']]
    #loop through coordinates 
    si = 0
    fixstart = False
    for i in range(1, len(x)):
        #calculate the gaze angle difference
        v1 = x[si]
        v2 = x[i]
        angle = abs(v1-v2)
        # check if the next coordinate is below maximal distance
        if angle <= maxdist and not fixstart:
        # start a new fixation
            si = 0 + i
            fixstart = True
            Sfix.append([time[i]])
        elif angle > maxdist and fixstart:
        # end the current fixation
            fixstart = False
            # only store the fixation if the duration is ok
            if time[i-1]-Sfix[-1][0] >= mindur:
                Efix.append((Sfix[-1][0], time[i-1], time[i-1]-Sfix[-1][0], bpx[si], bpy[si], x[si], block[si], nback[si],
                                target[si], zone[si], stimuli[si]))
            # delete the last fixation start if it was too short
            else:
                Sfix.pop(-1)
            si = 0 + i
        elif not fixstart:
            si += 1
    #add last fixation end (we can lose it if dist > maxdist is false for the last point)
    if len(Sfix) > len(Efix):
        Efix.append((Sfix[-1][0], time[len(x)-1], time[len(x)-1]-Sfix[-1][0], bpx[si], bpy[si], x[si], block[si], nback[si],
                                target[si], zone[si], stimuli[si]))
    return Efix